In [ ]:
#importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import gkey, username, password
import requests
import json
import geojson

In [ ]:
file_path = 'data/MSP_neighborhoods.csv'
MSP_df = pd.read_csv(file_path, encoding = "utf-8")

In [ ]:
MSP_df.head()

In [ ]:
# Convert age share columns to float and re-order into 5 different age groups
MSP_df['AgeBelow18Prct'] = (MSP_df['Under 5 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['5-9 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['10-14 years - Share'].str.rstrip('%').astype('float') / 100.0 
                              + MSP_df['15-17 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['Age18To34Prct'] = (MSP_df['18-24 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['25-34 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['Age35To54Prct'] = (MSP_df['35-44 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['45-54 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['Age55To75Prct'] = (MSP_df['55-64 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['65-74 years - Share'].str.rstrip('%').astype('float') / 100.0)

MSP_df['AgeAbove75Prct'] = (MSP_df['75-84 years - Share'].str.rstrip('%').astype('float') / 100.0
                              + MSP_df['85 years and older - Share'].str.rstrip('%').astype('float') / 100.0)

In [ ]:
# Set percent string fields to float fields
MSP_df['PublicTransportPrct'] = MSP_df['Public transportation - Share'].str.rstrip('%').astype('float') / 100.0
MSP_df['WalkBiketoWorkPrct'] = MSP_df['Walked, biked, worked at home, or other - Share'].str.rstrip('%').astype('float') / 100.0
MSP_df['UnemploymentPrct'] = MSP_df['Unemployment rate - Share'].str.rstrip('%').astype('float') / 100.0

In [ ]:
# Select columns to be used in our database
MSP_df_reduced = MSP_df[["Neighborhood", "City", "Total population",
    "Total households", "Median household income (2008-2012, 2012 dollars)", 
    "PublicTransportPrct", "WalkBiketoWorkPrct",
    "UnemploymentPrct", "AgeBelow18Prct", "Age18To34Prct", "Age35To54Prct",
    "Age55To75Prct", "AgeAbove75Prct"]]
MSP_df_reduced.head()

In [ ]:
# Rename MSP_df columns to match target database
neighborhood_df = MSP_df_reduced.rename(columns={"Total population": "NeighborhoodPopulation", 
                       "Total households": "NeighborhoodHouseholds",
                      "Median household income (2008-2012, 2012 dollars)": "MedianIncome"})
neighborhood_df.head()

In [ ]:
# Calculate the center of each neighborhood in Minneapolis 
# by averaging boundary coordinates
neighborhoodName = []
lats = []
lngs = []
avgLats = []
avgLngs = []
with open('data/Minneapolis_neighborhoods.geojson') as f:
    gj = geojson.load(f)
    
    # grab features from each neighborhood
    for i in gj['features']:
        
        #grab neighborhood name
        neighborhood = i['properties']['BDNAME']
        neighborhoodName.append(neighborhood)
        
        #clear lat and lngs lists for each loop
        lats.clear()
        lngs.clear()
        
        #grab coordinates that make up neighborhood bounds
        coordinates = i['geometry']['coordinates'][0][0]
        for i in coordinates:
            lats.append(i[1])
            lngs.append(i[0])
        
        #calculate average coordinates from list of coordinates and store
        avgLat = sum(lats) / len(lats)
        avgLng = sum(lngs) / len(lngs)
        avgLats.append(avgLat)
        avgLngs.append(avgLng)

In [ ]:
# repeat the above process for St. Paul
with open('data/StPaul_neighborhoods.geojson') as f:
    gj = geojson.load(f)
    
    # grab features from each neighborhood
    for i in gj['features']:
        
        #grab neighborhood name
        neighborhood = i['properties']['name2']
        neighborhoodName.append(neighborhood)
        
        #clear lat and lngs lists for each loop
        lats.clear()
        lngs.clear()
        
        #grab coordinates that make up neighborhood bounds
        coordinates = i['geometry']['coordinates'][0][0]
        for i in coordinates:
            lats.append(i[1])
            lngs.append(i[0])
        
        #calculate average coordinates from list of coordinates and store
        avgLat = sum(lats) / len(lats)
        avgLng = sum(lngs) / len(lngs)
        avgLats.append(avgLat)
        avgLngs.append(avgLng)

In [ ]:
neighborhood_center_dict = {"Neighborhood" : neighborhoodName,
                            "Latitude" :  avgLats,
                            "Longitude" : avgLngs
}

In [ ]:
center_coord_df = pd.DataFrame(neighborhood_center_dict)
center_coord_df

In [ ]:
# list of place types we will be looking for within google places
typeList = ['supermarket', 'park', 'gym', 'restaurant', 'school', 'transit_station', 'church']

In [ ]:
# create dictionary for storing data
places = {
    "placeName": [],
    "placeType": [],
    "placeLat": [],
    "placeLng": []
}
places

In [ ]:
for type in typeList:
    # parameters for api call
    params = {
        "radius": 3000,
        "type": type,
        "key": gkey
    }

    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # use iterrows to iterate through pandas dataframe
    for index, row in center_coord_df.iterrows():
        lat = row['Latitude']
        long = row['Longitude']
        location = f"{lat}, {long}"

        # add keyword to params dict
        params['location'] = location

        # assemble url and make API request
        places_data = requests.get(base_url, params=params).json()
        places_info = places_data['results']
        
        # iterate through results to pull out the name, lat, and long of found places
        for i in places_info:
            places['placeName'].append(i['name'])
            places['placeType'].append(type)
            places['placeLat'].append(i['geometry']['location']['lat'])
            places['placeLng'].append(i['geometry']['location']['lng'])

In [ ]:
places_df = pd.DataFrame(places)
places_df

In [ ]:
places_df.count()

In [ ]:
# Delete duplicate values which may have been grabbed by overlapping radius calls
places_no_dup_df = places_df.drop_duplicates()
places_no_dup_df.count()

In [ ]:
# Connect to database
connection_string = f"{username}:{password}@localhost:5432/MSP_Neighborhoods"

# Create the engine
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Checking connection by finding table names
engine.table_names()

In [ ]:
# Use "to_sql" function to load all transformed dfs' data into postgres

# Neighborhoods
neighborhood_df.to_sql(name='Neighborhoods', con=engine, if_exists='replace', index=True, index_label='NeighborhoodID')

# Places
places_no_dup_df.to_sql(name='Places', con=engine, if_exists='replace', index=True, index_label='placeID')